# Build a simple ML pipeline for image classification

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Create `Pipeline` with components

**Motivations** -This tutorial shows how to train a simple deep neural network using the [Fashion MNIST dataset and Keras on Azure Machine Learning. Fashion-MNIST is a dataset of Zalando's article images-consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.


# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [ ]:
# import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component

## 1.2 Configure credential
We are using `DefaultAzureCredential` to get access to workspace.

`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# Retrieve an already attached Azure Machine Learning Compute.
cpu_compute_target = "cpu-cluster"
print(ml_client.compute.get(cpu_compute_target))
gpu_compute_target = "gpu-cluster"
print(ml_client.compute.get(gpu_compute_target))

## 1.4 Prepare Job Input
By defining `Input`, you create a reference to the data source location. The data remains in its existing location, so no extra storage cost is incurred.

In [ ]:
from azure.ai.ml import Input

fashion_ds = Input(
    path="wasbs://demo@data4mldemo6150520719.blob.core.windows.net/mnist-fashion/"
)

In [ ]:
fashion_ds

# 2. Define and load command component
In this section, we will define and load component to build pipeline in two ways:
1. Using python function
1. Using yaml


## 2.1 Load components defined with python function
We define `Prep Data` component and `Train Image Classification Keras` component using @command_component respectively in [./prep/prep_component.py](./prep/prep_component.py) and [./train/train_component.py](./train/train_component.py).

Use following code to import component.

In [ ]:
%load_ext autoreload
%autoreload 2

# load component function from component python file
from prep.prep_component import prepare_data_component
from train.train_component import keras_train_component

# print hint of components
help(prepare_data_component)
help(keras_train_component)

(Optional) Python import will have cache if the code does not change. However, if you change the conda.yaml file, you will need to force re-import.

In [ ]:
# import importlib, prep.prep_component, train.train_component
# importlib.reload(prep.prep_component)
# importlib.reload(train.train_component)
# from prep.prep_component import prepare_data_component
# from train.train_component import keras_train_component

## 2.2 Load component defined with yaml
We define `Score Image Classification Keras` in [yaml](./score/score.yaml).

Use following code to load component from yaml.

In [ ]:
# load component function from yaml
keras_score_component = load_component(source="./score/score.yaml")

# 3. Build pipeline

We define a pipeline containing 3 nodes:
- `prepare_data_node` will load the image and labels from Fashion MNIST dataset into mnist_train.csv and mnist_test.csv. 
- `train_node` will train a CNN model with Keras using training data.
- `score_node` will score the model using test data.

In [ ]:
# define a pipeline containing 3 nodes: Prepare data node, train node, and score node
@pipeline(
    default_compute=cpu_compute_target,
)
def image_classification_keras_minist_convnet(pipeline_input_data):
    """E2E image classification pipeline with keras using python sdk."""
    prepare_data_node = prepare_data_component(input_data=pipeline_input_data)

    train_node = keras_train_component(
        input_data=prepare_data_node.outputs.training_data
    )
    train_node.compute = gpu_compute_target

    score_node = keras_score_component(
        input_data=prepare_data_node.outputs.test_data,
        input_model=train_node.outputs.output_model,
    )


# create a pipeline
pipeline_job = image_classification_keras_minist_convnet(pipeline_input_data=fashion_ds)

In [ ]:
print(pipeline_job)

# 4. Submit pipeline job

In [ ]:
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_samples"
)
pipeline_job

In [ ]:
# wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

# 5. (Optional) Register your component to workspace

If you want to share and reuse your component within workspace, you can also register it to a machine learning workspace.

The following sample code shows how to register a component to your workspace and get a registered component from your workspace.

In [ ]:
try:
    # try get back the component
    prep = ml_client.components.get(name="prep_data", version="1")
except:
    # if not exists, register component using following code
    prep = ml_client.components.create_or_update(prepare_data_component)

# list all components registered in workspace
for c in ml_client.components.list():
    print(c)

# Next Steps
You can see further examples of running a pipeline job [here](../)